In [1]:
import _pickle as pickle
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import timeit

from tqdm.notebook import tqdm

In [2]:
stats_keys = [
    "accuracy",
    "p_iwl",
    "context contains query class",
    "ic_pred",
    "iw_pred",
    "num p_iwl >= 0.5",
    "p_iwl given context contains query class",
    "loss"
    # "similarity",
]

# Transformer
stats_keys = [
    "accuracy",
    "context contains query class",
    "loss"
]

In [3]:
# repo_path = "/Users/chanb/research/ualberta/icl/simple_icl"
# results_dir = "/Users/chanb/research/ualberta/icl/scratch/evaluation_results"
# results_dir = "/Users/chanb/research/ualberta/icl/cc_results/evaluation_results"
# results_dir = "/Users/chanb/research/ualberta/icl/cc_results/noisy_labels_0.01/evaluation_results"
# results_dir = "/Users/chanb/research/ualberta/icl/cc_results/paper_experiments/evaluation_results"
# results_dir = "/Users/chanb/research/ualberta/icl/simple_icl/experiments/test_convergence_eval"

repo_path = "/Users/chanb/research/ualberta/icl/simple_icl"
results_dir = "/Users/chanb/research/ualberta/icl/cc_results/paper_experiments/evaluation_results"

# repo_path = "/home/chanb/src/simple_icl"
# results_dir = "/home/chanb/scratch/simple_icl/results"

repo_path = "/home/bryanpu1/projects/icl/simple_icl"
# results_dir = "/home/bryanpu1/projects/icl/scratch/evaluation_results"
results_dir = "/home/bryanpu1/projects/icl/cc_results/evaluation_results"

In [4]:
# variant_name = "synthetic-simple_icl"
# variant_name = "synthetic-transformer"
# variant_name = "synthetic-transformer-context_len"
# variant_name = "synthetic-transformer-large_num_low_freq"
# variant_name = "synthetic-transformer-large_num_low_freq-input_noise_0.2"
# variant_name = "synthetic-transformer-no_noise"
# variant_name = "synthetic-transformer-noisy_inputs_0.2"
# variant_name = "synthetic-transformer-noisy_labels_0.01"
# variant_name = "synthetic-transformer-noisy_labels_0.1"
# variant_name = "synthetic-transformer-num_relevant_contexts"

# variant_name = "omniglot-input_noise-larger_bs"
variant_name = "omniglot-input_noise"
variant_name = "omniglot-p_high"
# variant_name = "omniglot-p_relevant"
# variant_name = "omniglot-num_contexts"
# variant_name = "omniglot-num_relevant_contexts"

# variant_name = "synthetic-transformer-noisy_inputs"
# variant_name = "synthetic-transformer-noisy_labels"
# variant_name = "synthetic-transformer-num_contexts"
# variant_name = "synthetic-transformer-num_low_freq"
# variant_name = "synthetic-transformer-num_relevant_contexts"
# variant_name = "synthetic-transformer-input_noise-longer_epoch"
# variant_name = "synthetic-transformer-p_high"
# variant_name = "synthetic-transformer-p_relevant"

os.makedirs(
    os.path.join(repo_path, "plot_utils/plots/agg_stats"),
    exist_ok=True
)

In [5]:
stats_file = os.path.join(repo_path, "plot_utils/plots/agg_stats", "{}.feather".format(variant_name))

NUM_PARALLEL = 2

minibatch_count = 0
minibatch_content = ""
if os.path.isfile(stats_file):
    stats = pd.read_feather(stats_file)
else:
    stats = []

    if variant_name == "synthetic-simple_icl":
        all_dirs = [
            "synthetic-alpha",
            "synthetic-ic_predictor",
            "synthetic-iw_predictor",
        ]
    else:
        all_dirs = [variant_name]

    for curr_dir in all_dirs:
        curr_results_dir = os.path.join(results_dir, curr_dir)
        if repo_path.startswith("/Users"):
            it = tqdm(os.listdir(curr_results_dir))
        else:
            # it = os.listdir(curr_results_dir)
            it = tqdm(os.listdir(curr_results_dir))

        for run_name in it:
            tic = timeit.default_timer()

            if os.path.isdir(os.path.join(curr_results_dir, run_name)) and run_name.startswith("p_relevant_context"):
                # IC/IW
                for baseline_name in os.listdir(os.path.join(curr_results_dir, run_name)):
                    prefix = "-".join(baseline_name.split("-")[:-9])
                    seed = baseline_name.split("-")[-9]

                    data = pickle.load(open(os.path.join(curr_results_dir, run_name, baseline_name), "rb"))
                    for eval_name in data["stats"]:
                        for stats_key in stats_keys:
                            if not stats_key in data["stats"][eval_name]:
                                continue

                            factor = 1
                            if stats_key == "accuracy":
                                factor = 1 / 100

                            stats.append(
                                dict(
                                    model_type="iw" if "p_relevant_context_0.0" in baseline_name else "ic",
                                    dirname=curr_dir,
                                    variant=prefix,
                                    seed=seed,
                                    eval_name=eval_name,
                                    stats_key=stats_key,
                                    stats=[float(val) * factor for val in data["stats"][eval_name][stats_key]],
                                    **{
                                        "_".join(key_val.split("_")[:-1]): key_val.split("_")[-1] for key_val in prefix.split("-")
                                    },
                                )
                            )
                            stats[-1]["p_relevant_context"] = run_name.split("_")[-1]

            else:
                # Transformer
                if "p_relevant_context_0.0" in run_name or "p_relevant_context_1.0" in run_name:
                    continue

                prefix = "-".join(run_name.split("-")[:-9])
                seed = run_name.split("-")[-9]

                data = pickle.load(open(os.path.join(curr_results_dir, run_name), "rb"))

                minibatch_count += 1
                
                minibatch_content += "JAX_PLATFORMS=cpu python {}/plot_utils/simulate_minibatches.py --evaluation_file={} --repo_path={} --results_dir={} &\n".format(
                    repo_path,
                    os.path.join(curr_results_dir, run_name),
                    repo_path,
                    results_dir,
                )
                if minibatch_count % NUM_PARALLEL == 0:
                    minibatch_content += "wait \n"

                for eval_name in data["stats"]:
                    for stats_key in stats_keys:
                        if not stats_key in data["stats"][eval_name]:
                            continue

                        factor = 1
                        if stats_key == "accuracy":
                            factor = 1 / 100

                        stats.append(
                            dict(
                                model_type="transformer",
                                dirname=curr_dir,
                                variant=prefix,
                                seed=seed,
                                eval_name=eval_name,
                                stats_key=stats_key,
                                stats=[float(val) * factor for val in data["stats"][eval_name][stats_key]],
                                **{
                                    "_".join(key_val.split("_")[:-1]): key_val.split("_")[-1] for key_val in prefix.split("-")
                                },
                            )
                        )

            toc = timeit.default_timer()
    stats = pd.DataFrame(stats)
    stats.to_feather(os.path.join(repo_path, "plot_utils/plots/agg_stats", "{}.feather".format(variant_name)))
    minibatch_content += "wait \n"

with open("generate_fake_minibatches-{}.sh".format(variant_name), "w") as f:
    f.writelines(minibatch_content)

  0%|          | 0/19 [00:00<?, ?it/s]

In [6]:
len(stats)

3078

In [7]:
stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3078 entries, 0 to 3077
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   model_type          3078 non-null   object
 1   dirname             3078 non-null   object
 2   variant             3078 non-null   object
 3   seed                3078 non-null   object
 4   eval_name           3078 non-null   object
 5   stats_key           3078 non-null   object
 6   stats               3078 non-null   object
 7   dataset_size        3078 non-null   object
 8   p_relevant_context  3078 non-null   object
 9   p_high              3078 non-null   object
dtypes: object(10)
memory usage: 240.6+ KB


In [8]:
stats

,model_type,dirname,variant,seed,eval_name,stats_key,stats,dataset_size,p_relevant_context,p_high
0,transformer,omniglot-p_high,dataset_size_10000-p_relevant_context_0.9-p_hi...,seed_1,eval-default-none-flip_label,accuracy,"[0.0, 0.0, 0.08, 0.149, 0.174, 0.172, 0.169, 0...",10000,0.9,0.99
1,transformer,omniglot-p_high,dataset_size_10000-p_relevant_context_0.9-p_hi...,seed_1,eval-default-none-flip_label,context contains query class,"[0.898, 0.898, 0.898, 0.898, 0.898, 0.898, 0.8...",10000,0.9,0.99
2,transformer,omniglot-p_high,dataset_size_10000-p_relevant_context_0.9-p_hi...,seed_1,eval-default-none-flip_label,loss,"[7.751866817474365, 7.627061367034912, 8.47843...",10000,0.9,0.99
3,transformer,omniglot-p_high,dataset_size_10000-p_relevant_context_0.9-p_hi...,seed_1,eval-default-high_prob-flip_label,accuracy,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",10000,0.9,0.99
4,transformer,omniglot-p_high,dataset_size_10000-p_relevant_context_0.9-p_hi...,seed_1,eval-default-high_prob-flip_label,context contains query class,"[0.902, 0.902, 0.902, 0.902, 0.902, 0.902, 0.9...",10000,0.9,0.99
...,...,...,...,...,...,...,...,...,...,...
3073,transformer,omniglot-p_high,dataset_size_100000-p_relevant_context_0.9-p_h...,seed_1,eval-irrelevant_context-low_prob,context contains query class,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",100000,0.9,0.99
3074,transformer,omniglot-p_high,dataset_size_100000-p_relevant_context_0.9-p_h...,seed_1,eval-irrelevant_context-low_prob,loss,"[7.9077606201171875, 8.245863914489746, 7.7521...",100000,0.9,0.99
3075,transformer,omniglot-p_high,dataset_size_100000-p_relevant_context_0.9-p_h...,seed_1,pretraining,accuracy,"[0.0001, 0.9694, 0.9698000000000001, 0.9714, 0...",100000,0.9,0.99
3076,transformer,omniglot-p_high,dataset_size_100000-p_relevant_context_0.9-p_h...,seed_1,pretraining,context contains query class,"[0.9004, 0.9004, 0.9004, 0.9004, 0.9004, 0.900...",100000,0.9,0.99
